In [10]:
import pandas as pd
import xgboost as xgb

In [11]:
df_train = pd.read_csv("../Datasets/Boston/train.csv")
df_test = pd.read_csv("../Datasets/Boston/test.csv")

In [12]:
df_train.head()

,ID,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
3,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
4,7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.60,12.43,22.9


In [13]:
df_test.head()

,ID,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
0,3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03
1,6,0.02985,0.0,2.18,0,0.458,6.430,58.7,6.0622,3,222,18.7,394.12,5.21
2,8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311,15.2,396.90,19.15
3,9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311,15.2,386.63,29.93
4,10,0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5,311,15.2,386.71,17.10


In [14]:
df_train = df_train.drop(columns=["ID"])
df_test_index = df_test['ID']
df_test = df_test.drop(columns=["ID"])

In [15]:
from sklearn.model_selection import GridSearchCV
Y_train = df_train["medv"]
X_train = df_train.drop(columns=["medv"])

clf = xgb.XGBRFRegressor(objective="reg:squarederror", tree_method="hist", device="cuda", random_state=42)

param_grid = {
    "colsample_bynode": [0.6, 0.8, 1.0],
    "learning_rate": [0.5, 1],
    "max_depth": [4, 5, 6],
    "subsample": [0.6, 0.8]
}

grid_search = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    scoring="neg_root_mean_squared_error",
    cv=4,
    verbose=2,
    n_jobs=-1
)

grid_search.fit(X_train, Y_train)
clf = grid_search.best_estimator_

Fitting 4 folds for each of 36 candidates, totalling 144 fits


In [16]:
X_test = df_test
clf.predict(X_test)

array([36.143673 , 25.145462 , 16.49951  , 15.913993 , 17.424507 ,
       19.729548 , 20.50132  , 16.119274 , 16.115768 , 19.059137 ,
       19.70028  , 21.365854 , 16.664747 , 15.549736 , 22.147    ,
       21.32845  , 22.698181 , 27.952225 , 17.676481 , 24.319485 ,
       22.157764 , 23.571978 , 22.386642 , 21.69418  , 21.022741 ,
       22.557632 , 23.54497  , 23.33822  , 23.310957 , 26.754623 ,
       43.475563 , 42.915054 , 30.708666 , 20.26482  , 17.134865 ,
       20.510298 , 17.947678 , 16.99346  , 19.362064 , 19.87383  ,
       15.918743 , 20.099644 , 20.653933 , 16.355145 , 15.759844 ,
       14.661752 , 14.796305 , 15.619041 , 20.44046  , 20.613918 ,
       17.3735   , 30.856888 , 48.67301  , 20.831343 , 22.766087 ,
       23.605547 , 29.076847 , 47.990532 , 37.276436 , 32.740414 ,
       33.255135 , 45.077168 , 19.720808 , 17.714916 , 18.98671  ,
       20.946457 , 23.451227 , 22.871063 , 17.680443 , 21.197975 ,
       30.011538 , 44.57713  , 36.498085 , 37.55873  , 23.7059

In [17]:
answer = pd.DataFrame(clf.predict(X_test), index=df_test_index, columns=["medv"])
answer.index.name = "ID"


In [18]:
answer.to_csv("answer.csv")